In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.chdir('..')

In [2]:
# os.environ.get("ZHIPUAI_API_KEY")

# 设计和使用

## 核心设计

**关键概念**
- [x] focus概念：`<START>`, `<0>`, `<END>`
- [x] 提示语变量中包含历史对话，默认支持20轮次
- [x] 每个节点有独立的对话历史
- [x] 内容ID：在同一个对象中全局自增，如果重写会产生新的内容ID（这是为了与删除环不冲突）
- [ ] 删除环：每个修改命令将修改或删除前内容放入环形临时队列，以便恢复
- [ ] 恢复操作：不做回退，而是继续累加删除环
- [ ] 粘贴：先删除到删除环，再从修改环最后一个粘贴到指定位置

**指令模式**
- [ ] 创作是一个状态机，每个状态有一组当下可执行的命令
- [ ] 完整输入指令：`<focus> command prompt`
- [x] auto: 默认为`none`，将一步步询问；如果设为`all`则表示全自动生成
- [x] run 循环：进入一个输入和生成的对话循环
- [x] run 的 task 参数：直接将第一轮输入通过 task 参数给出
- [x] run 的 max_step 参数：执行最大步数后就退出（设置为1时可当作函数使用）
- [x] all指令：所有所有任务列表
- [x] todos指令：查看所有待办任务列表
- [ ] todo指令：重新生成指定项的扩写提纲或详细内容，默认为修改当前位置
- [x] ok指令：确认当前生成的内容为已完成，默认转移到下一个未完成项

**Web服务模式**
- [ ] 新项目对话
- [ ] 开启与对话绑定的创作
- [ ] 支持每一步指令的状态返回
- [ ] 支持流返回

**模型支持**
- [x] 支持手动创建LLM对象
- [x] 支持自动识别 ZHIPUAI_API_KEY 和 OPENAI_API_KEY，并自动加载包和构造LLM对象
- [ ] 支持文心一言、讯飞星火、通义千问等其他模型
- [ ] 支持私有部署大模型

**提示工程**
- 提示语中的提纲信息：
  - [x] 模式1：全路径选标题、扩写指南和内容摘要？
  - [ ] 模式2：祖先节点选标题，兄弟节点选标题、扩写指南和内容摘要，最近节点选文字详情？
- [ ] 提示语外部信息
- [ ] 提示语公共信息：在START节点配置
- [ ] 支持本地知识库检索
- [ ] 支持数据库查询
- [ ] 支持智能体询问

**导出模板**
- [ ] 导出提示语模板：包括任务提示、局部提示语等
- [ ] 导出提纲：导出JSON文件，包括文档树的标题、字数、扩写指南、内容摘要等

**导入模板**
- [ ] 导入提示语模板：从现有提示语目录中导入
- [ ] 导入提纲：从符合格式要求的JSON文件中
- [ ] 提纲仿写：从文件内容中提取出精确的仿写提纲
- [ ] 提纲优化：从文字内容中提取出段落摘要，反向拆解提纲，再给出优化建议

**导出成果**
- [ ] 文字合并：如何对待提纲中的标题，作为段落合并还是作为章节？
- [ ] 支持局部导出
- [ ] 支持仅导出大纲
- 按格式输出
  - [ ] markdown
  - [ ] word
  - [x] txt

**修改能力：**
- [ ] 删除：将内容及其子项剪切到删除环
- [ ] 排序：先剪切，再插入
- [ ] 重新生成提纲列表：若要重新生成提纲列表，必须手工清楚所有子项，否则禁止使用todo指令或对其修改
- [ ] todo指令：仅针对子项尚未完成的对象（如刚刚完成的，或其他兄弟节点）
- [ ] 修改提纲子项属性：需要单独使用title、words、howto等指令修改

**其他设计：**
- [ ] 支持对日志输出着色

## Bug

Bug清单：
- [x] START 记忆确实存在，但无法通过 START@memory 正确提取
- [x] ask 指令无法在当前节点重新生成
- [x] todos清单增加一个标记为当前任务的状态：`[当前任务]`, `* <未完成>`, `<已完成>`
- [x] is_completed 无法修改


# WritingProject

## 每一步都询问：使用默认的智谱AI

In [3]:
from langchain_chinese import WritingTask

# 使用默认的智谱AI推理
t = WritingTask()

In [4]:
t.step("致孩子的一封信, 800字")

-------------------- match --------------------
(None, '致孩子的一封信, 800字')
['move', 'todo', 'draft', 'all', 'task', 'ok']
default command: task
-------------------- invoke --------------------
<class 'langchain_chinese.writing.task.WritingTask'>
{'command': 'task', 'args': '致孩子的一封信, 800字'}
```json
{
    "总字数要求": 800,
    "标题名称": "致孩子的一封信：成长的寄语",
    "扩写指南": "本文是一封父母给予孩子的信，旨在表达对孩子的关爱、期许与鼓励。信中应涉及以下内容：回顾孩子成长的点滴，表达父母的爱与感激；针对孩子的个性特点提出建设性建议；分享人生经验，引导孩子树立正确价值观；最后，表达对孩子未来的美好祝愿。信中可提及孩子的兴趣爱好、父母与孩子的共同回忆，以及一些具体的生活例子。"
}
```


{'command': 'task',
 'args': '致孩子的一封信, 800字',
 'reply': {'总字数要求': 800,
  '标题名称': '致孩子的一封信：成长的寄语',
  '扩写指南': '本文是一封父母给予孩子的信，旨在表达对孩子的关爱、期许与鼓励。信中应涉及以下内容：回顾孩子成长的点滴，表达父母的爱与感激；针对孩子的个性特点提出建设性建议；分享人生经验，引导孩子树立正确价值观；最后，表达对孩子未来的美好祝愿。信中可提及孩子的兴趣爱好、父母与孩子的共同回忆，以及一些具体的生活例子。'}}

In [6]:
t.step("ok")

-------------------- match --------------------
('ok', '')
['move', 'todo', 'draft', 'all', 'task', 'ok']
default command: task
-------------------- invoke --------------------
<class 'langchain_chinese.writing.task.WritingTask'>
{'command': 'ok', 'args': ''}
<#init> ok


{'command': 'ok', 'args': '', 'reply': 'todo'}

In [7]:
t.step("task 继续")

-------------------- match --------------------
('task', '继续')
['move', 'todo', 'draft', 'all', 'title', 'words_advice', 'howto', 'summarise', 'text', 'task', 'ok']
default command: task
-------------------- invoke --------------------
<class 'langchain_chinese.writing.task.WritingTask'>
{'command': 'task', 'args': '继续'}
```json
{
  "大纲列表": [
    {
      "总字数要求": 250,
      "标题名称": "爱的表白",
      "扩写指南": "描述对孩子的独特个性的欣赏，以及父母对孩子的深深的爱意和感激之情。可以提及孩子的一个特别的行为或兴趣，强调这是孩子独有的，让父母感到骄傲的特质。"
    },
    {
      "总字数要求": 250,
      "标题名称": "家庭智慧",
      "扩写指南": "分享家庭的核心价值观和人生哲理，通过一些简单的故事或成语，如‘水滴石穿’、‘千里之行始于足下’，来阐述这些价值观和哲理的重要性，鼓励孩子在日常生活中实践它们。"
    },
    {
      "总字数要求": 200,
      "标题名称": "坚韧的力量",
      "扩写指南": "用父母自己的经历或寓言故事，如‘愚公移山’，来鼓励孩子在遇到挑战时保持坚韧不拔的精神，告诉他们困难是成长的磨石。"
    },
    {
      "总字数要求": 100,
      "标题名称": "珍惜每分每秒",
      "扩写指南": "提醒孩子时间的宝贵，鼓励他们珍惜每一刻，勇敢追求自己的梦想，不要畏惧失败，因为‘失败是成功之母’。"
    }
  ]
}
```


{'command': 'task',
 'args': '继续',
 'reply': {'大纲列表': [{'总字数要求': 250,
    '标题名称': '爱的表白',
    '扩写指南': '描述对孩子的独特个性的欣赏，以及父母对孩子的深深的爱意和感激之情。可以提及孩子的一个特别的行为或兴趣，强调这是孩子独有的，让父母感到骄傲的特质。'},
   {'总字数要求': 250,
    '标题名称': '家庭智慧',
    '扩写指南': '分享家庭的核心价值观和人生哲理，通过一些简单的故事或成语，如‘水滴石穿’、‘千里之行始于足下’，来阐述这些价值观和哲理的重要性，鼓励孩子在日常生活中实践它们。'},
   {'总字数要求': 200,
    '标题名称': '坚韧的力量',
    '扩写指南': '用父母自己的经历或寓言故事，如‘愚公移山’，来鼓励孩子在遇到挑战时保持坚韧不拔的精神，告诉他们困难是成长的磨石。'},
   {'总字数要求': 100,
    '标题名称': '珍惜每分每秒',
    '扩写指南': '提醒孩子时间的宝贵，鼓励他们珍惜每一刻，勇敢追求自己的梦想，不要畏惧失败，因为‘失败是成功之母’。'}]}}

In [8]:
t.step("ok")

-------------------- match --------------------
('ok', '')
['move', 'todo', 'draft', 'all', 'title', 'words_advice', 'howto', 'summarise', 'text', 'task', 'ok']
default command: task
-------------------- invoke --------------------
<class 'langchain_chinese.writing.task.WritingTask'>
{'command': 'ok', 'args': ''}
<#todo> ok


{'command': 'ok', 'args': '', 'reply': 'done'}

## 全自动生成：使用默认的智谱AI